## Data Preprocessing Tools

Importing the libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pls

Importing the dataset

In [4]:
dataset = pd.read_csv("Data.csv")
# Xler bizim feature vektörümüz olurken y bizim dependent veriablemiz olacak
# dataset.iloc[:,:] ilk : alınacak row sayısını ifade ederken ikinci : alınacak column sayısını ifade eder
# Burada bütün verileri almak isteyip sadece verilerimizi columna göre bölmek istediğimiz için ilk : olduğu gibi bırakıyoruz
# : olduğu gibi bırakmak demek hepsini al demektir.
# .values eki sadece verileri al demek yani column nameleri ve row indexleri aradan çıkarılır.
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [5]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


In [6]:
print(y)

['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


In [7]:
missing_values = dataset.isna().sum()

In [8]:
print(missing_values)

Country      0
Age          1
Salary       1
Purchased    0
dtype: int64


Taking care of missing data

In [9]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy='mean')

In [10]:
imputer.fit(X[:,1:3])
X[:,1:3] = imputer.transform(X[:,1:3])

In [11]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


Encoding categorical data

Encoding the Independent Variable

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [13]:
# transformers argümanı içerisinde 3 değer olan bir tuple alacaktır
# [('transformerimize vermek istediğimiz isim (herhangi bir şey olabilir)'),Kullanacağımız encoder vey columntransformer,[uygulanacağı index]]
# remainder = 'passthrough' olarak bırakılmasının sebebi dönüşüm yapılmayacak sütünların olduğu gibi kalmasını sağlamaktır.
ct = ColumnTransformer(transformers = [('encoder',OneHotEncoder(),[0])], remainder = 'passthrough')
# fit_transform kodu ayarladığımız dönüşümleri istediğimiz veri setine uygular ve işlenmiş veri setini döndürür
# yanlız veri setinin geri dönüş tipi np arrayi olmadığından outputu np.array() içine alarak np arrayi olmaya zorlarız.
X = np.array(ct.fit_transform(X))

In [14]:
print(X)

[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


Encoding the Dependent Variable

In [15]:
print(y)

['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


In [16]:
# Yukarıda da görüldüğü üzere y verisetimiz matris değil vektör şeklindedir.
# Bu yüzden y verisetimizi label olarak tanımlarız
# Labelleri encode etmek için başka bir fonksiyon kullanırız
# Ayrıca bu şekilde kullanabilmemizin sebebi sadece 2 classa sahip olmasıdır
from sklearn.preprocessing import LabelEncoder

In [17]:
le = LabelEncoder()
y = le.fit_transform(y)

In [18]:
print(y)

[0 1 0 0 1 1 0 1 0 1]


Splitting the dataset into the Training set and Test set

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [20]:
print(X_train)

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]


In [21]:
print(X_test)

[[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


In [22]:
print(y_train)

[0 1 0 0 1 1 0 1]


In [23]:
print(y_test)

[0 1]


Feature Scaling

In [25]:
# X_train'de kullandığımız fit_transform metodunu X_test'de kullanamayız
# Çünkü fit scalerimizi eğitirken transform eğitilmiş scaleri verisetimize uygular
# Test setimiz de aynı scaler ile scale edilmek zorundadır
# Bu yüzden eğitilmiş scalerimizi transform metoduyla X_test verimize uygularız
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:,3:] = sc.fit_transform(X_train[:,3:])
X_test[:,3:] = sc.transform(X_test[:,3:])

In [28]:
print(X_train)

[[0.0 0.0 1.0 -0.19159184384578545 -1.0781259408412425]
 [0.0 1.0 0.0 -0.014117293757057777 -0.07013167641635372]
 [1.0 0.0 0.0 0.566708506533324 0.633562432710455]
 [0.0 0.0 1.0 -0.30453019390224867 -0.30786617274297867]
 [0.0 0.0 1.0 -1.9018011447007988 -1.420463615551582]
 [1.0 0.0 0.0 1.1475343068237058 1.232653363453549]
 [0.0 1.0 0.0 1.4379472069688968 1.5749910381638885]
 [1.0 0.0 0.0 -0.7401495441200351 -0.5646194287757332]]


In [27]:
print(X_test)

[[0.0 1.0 0.0 -1.4661817944830124 -0.9069571034860727]
 [1.0 0.0 0.0 -0.44973664397484414 0.2056403393225306]]
